# Compaction Project

## Problem Formulation

Given a set of chunks whose sizes are

$$
S_n = \{d_1, \cdots, d_n\},
$$

where the positive integer $d_i \leq 2048$ for all $i = 1, \cdots, n$. Suppose the remaining operators need time 

$$
f(d_i) = C_1 + d_i \times C_2
$$

to process a data chunk with the size $d_i$. Our goal is to compact the set $S$, i.e., we need a transformation

$$
\mathcal{M}: S_n \rightarrow S'_m \triangleq \{d'_1,  \cdots, d'_m\},
$$

where $\sum_i^n d_i = \sum_j^m d'_j$ and $m$ is an arbitrary integer less than $n$, to minimize 

$$
\sum_j^m f(d'_j) + cost(M, S).
$$

where $cost(\mathcal{M}, S)$ is the cost of the transformation $\mathcal{M}$ on the set $S$. 

The cost of combining two or more chunks into one: $d_i + \cdots + d_j = d'_s \leq 2048$, is 

$$
g(d'_s) = C_3 + d'_s \times C_4.
$$

**Note:** This formulated problem is easier than the real compaction problem because we have the sizes of all data chunks in advance, rather than a chunk stream.

In [23]:
# utils
from termcolor import colored

def print_color(text, color='black'):
    print(colored(text, color))

## 1. Chunk Sizes Distribution

In [24]:
import numpy as np

# Generate random chunk sizes from a Gaussian distribution
def generate_chunk_sizes(n, mean=64, scale=256):
    return np.minimum(2048, np.maximum(1, np.random.normal(mean, scale, n))).astype(int)

## 2. Compaction Simulator

In [25]:
#             fixed cost      per tuple cost
# probe()     1.5             0.03
# next()      0.9             0.06
# --------------------------------------
# compact()   0.3             0.03
# --------------------------------------

k_prc_fixed_cost = (1.5 + 0.9)
k_prc_per_tuple_cost = (0.03 + 0.06)
k_cpt_fixed_cost = 0.3
k_cpt_per_tuple_cost = 0.03

# This function split each chunk into smaller chunks.
def split_array_np(numbers, parts):
    numbers = np.array(numbers)

    quotient, remainder = np.divmod(numbers, parts)
    split = np.repeat(quotient, parts)
    remainder = np.repeat(remainder, parts)
    split[np.arange(numbers.size * parts) % parts < remainder] += 1
        
    split = split[split > 0]

    return split

# Simulate the execution of joins.
def simulate_join(sizes, compact_func, chunk_factor=1, n_join=1, print_log=True):
    prc_cost = 0
    cpt_cost = 0
    next_sizes = np.array(sizes)

    if print_log:
        print_color(f"-------------------------", 'green')
        print_color(f"Compactor {compact_func}", 'green')

    for l in reversed(range(n_join)):
        # 1. Join
        prc_cost += k_prc_fixed_cost * len(next_sizes) + np.sum(next_sizes) * k_prc_per_tuple_cost

        # 2. Split chunks
        next_sizes = split_array_np(next_sizes, chunk_factor)
        input_chunk_number = len(next_sizes)

        # 3. Compact
        next_sizes, cost = compact_func(next_sizes, chunk_factor, l)
        output_chunk_number = len(next_sizes)

        if print_log:
            print_color(f"Level {n_join - l}: {input_chunk_number} -> {output_chunk_number} chunks, cost: {cost:.2f}", 'green')

        cpt_cost += cost

    return prc_cost, cpt_cost

## 3. Compaction Strategies

### Cost Calculation

In [26]:
def compute_prc_cost(chunk_size, chunk_factor, n_joins):
    per_tuple_cost = k_prc_per_tuple_cost * chunk_size * n_joins

    fixed_cost = 0
    for i in range(n_joins):
        fixed_cost += k_prc_fixed_cost * np.minimum(chunk_factor ** i, chunk_size)
    
    # print(f"per_tuple_cost: {per_tuple_cost:.2f}, fixed_cost: {fixed_cost:.2f}")
    return per_tuple_cost + fixed_cost


# Compute the cost of a single compaction
def compute_cpt_cost(sizes_in_one_compaction):
    return k_cpt_fixed_cost + np.sum(sizes_in_one_compaction) * k_cpt_per_tuple_cost

### Base Strategies

**Strategy 1**: Do not compact any chunks.

**Strategy 2**: Fully compact all chunks.

In [27]:
def alg_no_compaction(chunk_sizes, chunk_factor, level):
    return chunk_sizes, 0


def alg_full_compaction(chunk_sizes, chunk_factor, level):
    transformed_sizes = []
    cpt_cost = 0
    cpt_sizes = []
    
    for size in chunk_sizes:
        if size == 2048: 
            transformed_sizes.append(size)
            continue

        if sum(cpt_sizes) + size <= 2048:
            cpt_sizes.append(size)
        else:
            cpt_cost += compute_cpt_cost(cpt_sizes)
            transformed_sizes.append(sum(cpt_sizes))
            cpt_sizes = [size]
    
    if cpt_sizes:
        cpt_cost += compute_cpt_cost(cpt_sizes)
        transformed_sizes.append(sum(cpt_sizes))

    return transformed_sizes, cpt_cost

### Optimal Strategy

**Strategy 3**: Sort all chunks ascendingly and compact them in order, until compaction is not beneficial

In [28]:
def alg_sort_compaction(chunk_sizes, chunk_factor, n_joins):
    assert len(chunk_sizes) > 0, "chunk_sizes must not be empty"

    sorted_sizes = sorted(chunk_sizes)
    transformed_sizes = []
    cpt_cost = 0

    i = 0
    cpt_sizes = [sorted_sizes[0]]
    for i in range(1, len(sorted_sizes), 1):
        size = sorted_sizes[i]
        cur_sum = sum(cpt_sizes)

        if cur_sum + size <= 2048:
            gain = compute_prc_cost(cur_sum, chunk_factor, n_joins) + compute_prc_cost(size, chunk_factor, n_joins) - compute_prc_cost(cur_sum + size, chunk_factor, n_joins)
            loss = (k_cpt_fixed_cost + size * k_cpt_per_tuple_cost)
            
            # print(f"gain: {gain:.2f}, loss: {loss:.2f}")
            # print(f"cur sum {cur_sum} + size {size} = {cur_sum + size}")

            if gain - loss > 0:
                cpt_sizes.append(size)
            else:
                break
        else:
            if len(cpt_sizes) > 1:
                cpt_cost += compute_cpt_cost(cpt_sizes)
            transformed_sizes.append(sum(cpt_sizes))

            cpt_sizes = [size]

    if cpt_sizes:
        if len(cpt_sizes) > 1:
            cpt_cost += compute_cpt_cost(cpt_sizes)
        transformed_sizes.append(sum(cpt_sizes))

    # Append the remaining chunks, it is not beneficial to compact them
    for j in range(i+1, len(sorted_sizes)):
        transformed_sizes.append(sorted_sizes[j])
        
    return transformed_sizes, cpt_cost

### DuckDB Strategy

**Strategy 4**: Set a threhold to distinguish the big chunk and the small chunk, we only compact small chunks.

In [29]:
k_block_size = 2048
k_duckdb_compaction_threshold = 128

def alg_binary_compaction(chunk_sizes, chunk_factor, level):
    trans_chunks = []
    cpt_cost = 0
    
    cpt_chunks = []
    for size in chunk_sizes:
        if size >= k_duckdb_compaction_threshold:
            trans_chunks.append(size)
            continue

        cpt_chunks.append(size)

        if sum(cpt_chunks) >= k_block_size - k_duckdb_compaction_threshold:
            trans_chunks.append(np.sum(cpt_chunks))
            cpt_cost += compute_cpt_cost(cpt_chunks)
            cpt_chunks = []

    
    return trans_chunks, cpt_cost

## 4. Let's Join!

In [30]:
chunk_sizes = generate_chunk_sizes(n=int(1e7 / 2048), mean=2048, scale=128)
chunk_factor = 16
num_join = 3
print_log = False

grades = {
    "No Compaction": simulate_join(chunk_sizes, alg_no_compaction, chunk_factor, num_join, print_log), 
    "Full Compaction": simulate_join(chunk_sizes, alg_full_compaction, chunk_factor, num_join, print_log),
    "Sort Compaction": simulate_join(chunk_sizes, alg_sort_compaction, chunk_factor, num_join, print_log),
    "Binary Compaction": simulate_join(chunk_sizes, alg_binary_compaction, chunk_factor, num_join, print_log), 
}

print_color(f"-------------------------", 'green')

for grade in grades:
    prc_cost = grades[grade][0]/1e6
    cpt_cost = grades[grade][1]/1e6
    print_color(f"[{grade}]", 'green')
    print(f"\t Total Cost: {prc_cost + cpt_cost:.2f}s\tCompute Cost: {prc_cost:.2f}s\t Compaction Cost: {cpt_cost:.2f}s")

-------------------------
[No Compaction]
	 Total Cost: 5.83s	Compute Cost: 5.83s	 Compaction Cost: 0.00s
[Full Compaction]
	 Total Cost: 3.55s	Compute Cost: 2.67s	 Compaction Cost: 0.88s
[Sort Compaction]
	 Total Cost: 3.13s	Compute Cost: 2.84s	 Compaction Cost: 0.29s
[Binary Compaction]
	 Total Cost: 3.48s	Compute Cost: 2.76s	 Compaction Cost: 0.72s
